In [1]:
import tensorflow as tf
import os
import numpy as np
import pickle
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
input_layer = tf.keras.layers.Input((32, 3))
lstm_1 = tf.keras.layers.LSTM(16, activation="tanh", recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(input_layer)
lstm_2 = tf.keras.layers.LSTM(64, activation="tanh", recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_1[0])
lstm_3 = tf.keras.layers.LSTM(256, activation="tanh", recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_2[0])
lstm_4 = tf.keras.layers.LSTM(512, activation="tanh", recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_3[0])
lstm_5 = tf.keras.layers.LSTM(256, activation="tanh", recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_4[0])
lstm_6 = tf.keras.layers.LSTM(64, activation="tanh", recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_5[0])
lstm_7 = tf.keras.layers.LSTM(16, activation="tanh", recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_6[0])
lstm_8 = tf.keras.layers.LSTM(2, activation="tanh", recurrent_activation='sigmoid', dropout = 0.15, return_sequences = True, return_state = True)(lstm_7[0])
print(lstm_8[0])

KerasTensor(type_spec=TensorSpec(shape=(None, 32, 2), dtype=tf.float32, name=None), name='lstm_7/PartitionedCall:1', description="created by layer 'lstm_7'")


In [3]:
model = tf.keras.models.Model(inputs = input_layer, outputs = lstm_8[0], name = "DLVS_Model")

In [4]:
import os
error_final = []
vel_final = []
path = "/home/uas-dtu/Documents/DL_IBVS/Data/"
for a in os.listdir(path):
    file = open(path + "/" + a, "r")
    for i in [file.readlines()]:
        for j in range(len(i) - 32):
#             print(j)
            error_data = []
            vel_data = []
            for k in range(32):
                error_data.append([float(i[j + k].split(",")[0]), 
                                   float(i[j + k].split(",")[1]),
                                   float(i[j + k].split(",")[2])])
                vel_data.append([float(i[j + k].split(",")[3]), 
                                 float(i[j + k].split(",")[4])])
            error_final.append(error_data)
            vel_final.append(vel_data)

In [5]:
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence
import tensorflow as tf
import os
import json
import random


class DataGenerator(Sequence):
    def __init__(self, error_final, vel_final,
                 batch_size=32,
                 shuffle=True):
        self.error_final = error_final
        self.vel_final = vel_final
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
        if self.shuffle:
            temp = [i for i in range(len(self.error_final))]
            random.shuffle(temp)
            #       self.flow_videos = [self.flow_videos[i] for i in temp]
            self.error_final = [self.error_final[i] for i in temp]
            self.vel_final = [self.vel_final[i] for i in temp]

    def __len__(self):
        return int(np.floor(len(self.error_final) / self.batch_size))


    def __getitem__(self, index):
        start_index = index * self.batch_size
#         input_1 = []
        error_reading = []
        vel_reading = []
        i = start_index - 1
        while len(error_reading) < self.batch_size:
            try:
                i += 1
                error = self.error_final[i%len(self.error_final)]
                vel = self.vel_final[i%len(self.vel_final)]
                error_reading.append(error)
                vel_reading.append(vel)

            except Exception as err:
                print(err)
                continue
            
        vel_reading = np.array(vel_reading, dtype = np.float32)
        error_reading = np.array(error_reading, dtype = np.float32)
            
        return error_reading, vel_reading

In [20]:
train_data = DataGenerator(error_final, vel_final, batch_size=1, shuffle=False)

In [21]:
### Testing ###

In [22]:
model.load_weights("./NILU_TRAIN/train_70.tf")

In [23]:
pred = model.predict(train_data.__getitem__(0)[0])
print(train_data.__getitem__(0)[1])
print(pred)

[[[0.01 0.01]
  [0.01 0.01]
  [0.01 0.01]
  [0.01 0.01]
  [0.01 0.01]
  [0.01 0.01]
  [0.01 0.01]
  [0.01 0.01]
  [0.01 0.01]
  [0.01 0.01]
  [0.01 0.01]
  [0.04 0.23]
  [0.04 0.23]
  [0.04 0.23]
  [0.04 0.23]
  [0.04 0.23]
  [0.04 0.23]
  [0.04 0.23]
  [0.04 0.23]
  [0.04 0.23]
  [0.09 0.78]
  [0.09 0.78]
  [0.09 0.78]
  [0.09 0.78]
  [0.09 0.78]
  [0.09 0.78]
  [0.09 0.78]
  [0.09 0.78]
  [0.09 0.78]
  [0.09 0.78]
  [0.09 0.78]
  [0.09 0.78]]]
[[[0.04889322 0.7605096 ]
  [0.09317811 0.96371317]
  [0.1037365  0.994967  ]
  [0.11744983 0.9993013 ]
  [0.12521642 0.99989784]
  [0.12996484 0.9999816 ]
  [0.13358606 0.99999386]
  [0.13670011 0.999996  ]
  [0.1398925  0.99999654]
  [0.14356855 0.9999968 ]
  [0.14769818 0.9999969 ]
  [0.1518993  0.99999684]
  [0.15583989 0.999997  ]
  [0.15927538 0.999997  ]
  [0.1619829  0.9999969 ]
  [0.16441086 0.9999969 ]
  [0.16662708 0.99999696]
  [0.16857767 0.99999696]
  [0.17028543 0.9999969 ]
  [0.17164165 0.9999969 ]
  [0.17264761 0.9999969 ]
  [0